In [1]:
from vr.api.request_vr_api import get_vr

dados = get_vr(latitude="-2.46587780311219", longitude="-48.2928049787397")
dados

{'cnpj': '48409628000171', 'nomeFantasia': 'AGRO PEC', 'produto': 'Refei', 'tipoEnd': 'R', 'endereco': ' NOVA', 'numero': '4', 'bairro': 'QUATRO BOCAS', 'cidade': 'TOM', 'estado': 'PA', 'cep': '68680-000', 'ddd': '091', 'telefone': '993108628', 'latitude': -2.41832, 'longitude': -48.1566, 'flIfood': None, 'flRappi': None, 'flUbereats': None, 'flVrpague': None, 'flDelivery': None, 'dtInclusao': '2023-04-03', 'produtos': ['Alimenta', 'Cultura', 'Refei']}
{'cnpj': '30545116000177', 'nomeFantasia': 'MERCANTIL SALES', 'produto': 'Refei', 'tipoEnd': 'R', 'endereco': ' BERNARDO CURY', 'numero': '24', 'bairro': 'TABOM', 'cidade': 'TOM', 'estado': 'PA', 'cep': '68680-000', 'ddd': '091', 'telefone': '92435091', 'latitude': -2.4164, 'longitude': -48.1631, 'flIfood': None, 'flRappi': None, 'flUbereats': None, 'flVrpague': None, 'flDelivery': None, 'dtInclusao': '2023-04-03', 'produtos': ['Alimenta', 'Refei']}
{'cnpj': '08229013000149', 'nomeFantasia': 'PANIFICADORA GLEYCE', 'produto': 'Refei', 'ti

[{'CNPJ': '48409628000171',
  'RAZAO_SOCIAL': None,
  'ESTABELECIMENTOS': 'AGRO PEC',
  'ENDERECO': 'R NOVA ',
  'BAIRRO': 'QUATRO BOCAS',
  'CIDADE': 'TOM',
  'UF': 'PA',
  'CEP': '68680-000',
  'TELEFONE': '091, 993108628',
  'EMAIL': None,
  'LATITUDE': -2.41832,
  'LONGITUDE': -48.1566,
  'BANDEIRA': 'VR'},
 {'CNPJ': '30545116000177',
  'RAZAO_SOCIAL': None,
  'ESTABELECIMENTOS': 'MERCANTIL SALES',
  'ENDERECO': 'R BERNARDO CURY ',
  'BAIRRO': 'TABOM',
  'CIDADE': 'TOM',
  'UF': 'PA',
  'CEP': '68680-000',
  'TELEFONE': '091, 92435091',
  'EMAIL': None,
  'LATITUDE': -2.4164,
  'LONGITUDE': -48.1631,
  'BANDEIRA': 'VR'},
 {'CNPJ': '08229013000149',
  'RAZAO_SOCIAL': None,
  'ESTABELECIMENTOS': 'PANIFICADORA GLEYCE',
  'ENDERECO': 'PCDO MERCADO ',
  'BAIRRO': 'QUATRO BOCAS',
  'CIDADE': 'TOM',
  'UF': 'PA',
  'CEP': '68682-000',
  'TELEFONE': '091, 93175599',
  'EMAIL': None,
  'LATITUDE': -2.40997,
  'LONGITUDE': -48.2259,
  'BANDEIRA': 'VR'},
 {'CNPJ': '16647293000133',
  'RAZAO_S

In [ ]:
import re
retorno = re.findall("^[0-9]{2}([^@]+)","952345SA#124@APNNE")
retorno

In [ ]:
import json
import requests
import boto3
import ftplib
import patoolib
import botocore
import pandas as pd
import os
from datetime import date
import logging
import traceback
from zipfile import ZipFile
from dateutil import rrule
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta



assume_role_cache: dict = {}
def assumed_role_session(role_arn: str, base_session: botocore.session.Session = None):
    base_session = base_session or boto3.session.Session()._session
    fetcher = botocore.credentials.AssumeRoleCredentialFetcher(
        client_creator = base_session.create_client,
        source_credentials = base_session.get_credentials(),
        role_arn = role_arn,
        extra_args = {
        #    'RoleSessionName': None # set this if you want something non-default
        }
    )
    creds = botocore.credentials.DeferredRefreshableCredentials(
        method = 'assume-role',
        refresh_using = fetcher.fetch_credentials,
        time_fetcher = lambda: datetime.datetime.now(tzlocal())
    )
    botocore_session = botocore.session.Session()
    botocore_session._credentials = creds
    return boto3.Session(botocore_session = botocore_session)


# usage:
session = assumed_role_session('arn:aws:iam::274674695952:role/service-role/LambdaRole')


s3 = boto3.client('s3')

# Variaveis
S3_BUCKET_NAME = 'abrasel-datalake'

PATH_FOLDER = '/tmp/'

s3_resource = boto3.resource('s3')
bucket = s3_resource.Bucket(S3_BUCKET_NAME)

url_base = 'http://200.152.38.155/CNPJ/'
OUTPUT_DIR = 's3://abrasel-datalake/raw/CNPJ/cnpj_estabele/'

def extract_file(url, file_name):
    # faz requisição ao servidor

    with requests.get(url, stream=True) as response:
        if response.status_code == requests.codes.OK:
            with open(file_name, 'wb') as f:
                for chunk in response.iter_content(chunk_size=32768):
                    f.write(chunk)
            print(f"Download finalizado em {file_name}")
        else:
            response.raise_for_status()       

    """
    resposta = requests.get(url)
    if resposta.status_code == requests.codes.OK:
        with open(file_name, 'wb') as cnpj_estabelecimento:
            cnpj_estabelecimento.write(resposta.content)
        print(f"Download finalizado em {file_name}")
    else:
        resposta.raise_for_status()
    """
        

def lambda_handler(event, context):
    #Lista os objetos dentro do bucket
    response = s3.list_objects_v2(Bucket=S3_BUCKET_NAME, Prefix="raw/CNPJ/cnpj_estabele/")
    try:
        files_in_folder = response["Contents"]
        for file in files_in_folder:
            name_to_delete = [{'Key': file['Key']}]
            # Deleta os objetos dentro do bucket
            s3.delete_objects(Bucket=S3_BUCKET_NAME, Delete={'Objects': name_to_delete})
    except:
        pass
    
    for i in range(0,5):
        BASE_URL = f'{url_base}Estabelecimentos{i}.zip'
        name_file = f'Estabelecimentos{i}.zip'
        full_path = os.path.join(PATH_FOLDER,name_file)
        extract_file(url=BASE_URL,file_name=full_path)
        logging.info(full_path)
        logging.info("Download finalizado in /tmp/..")
        with ZipFile(full_path, "r") as zip_code:
            zip_code.extractall(PATH_FOLDER)
        print(zip_code.namelist())
        
        print("Dados descompactados..")
        
        name_unzip = zip_code.namelist()
        for name in name_unzip:
            print(name)
            path_unzip = f'{PATH_FOLDER}{name}'
            #bucket.upload_file(path_unzip, f"raw/CNPJ/cnpj_estabele/{name}")
            s3.upload_file(f"{path_unzip}", S3_BUCKET_NAME, f"raw/CNPJ/cnpj_estabele/{name}")
        print("Download Finalizado..")
        
            

    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }


In [ ]:
import json
import requests
import boto3
import os
import logging
from zipfile import ZipFile
import botocore
from datetime import datetime
from dateutil.tz import tzlocal

# Variáveis
S3_BUCKET_NAME = 'abrasel-datalake'
PATH_FOLDER = '/tmp/'
url_base = 'http://200.152.38.155/CNPJ/'
OUTPUT_DIR = 's3://abrasel-datalake/raw/CNPJ/cnpj_estabele/'

# Criação da sessão com função IAM assumida
def assumed_role_session(role_arn: str, base_session: botocore.session.Session = None):
    base_session = base_session or boto3.session.Session()._session
    fetcher = botocore.credentials.AssumeRoleCredentialFetcher(
        client_creator = base_session.create_client,
        source_credentials = base_session.get_credentials(),
        role_arn = role_arn,
        extra_args = {
            # 'RoleSessionName': None # Defina isso se desejar algo não padrão
        }
    )
    creds = botocore.credentials.DeferredRefreshableCredentials(
        method = 'assume-role',
        refresh_using = fetcher.fetch_credentials,
        time_fetcher = lambda: datetime.now(tzlocal())
    )
    botocore_session = botocore.session.Session()
    botocore_session._credentials = creds
    return boto3.Session(botocore_session = botocore_session)

# usage:
session = assumed_role_session('arn:aws:iam::274674695952:role/service-role/LambdaRole')

s3 = boto3.client('s3')

# Criação de recurso usando a sessão assumida
s3_resource = session.resource('s3')
bucket = s3_resource.Bucket(S3_BUCKET_NAME)

def extract_and_upload_file(url, file_name):
    try:
        with requests.get(url, stream=True) as response:
            with open(file_name, 'wb') as f:
                for chunk in response.iter_content(chunk_size=32768):
                    f.write(chunk)
            logging.info(f"Download finalizado em {file_name}")

            with ZipFile(file_name, "r") as zip_code:
                zip_code.extractall(PATH_FOLDER)
                extracted_files = zip_code.namelist()

                for extracted_file in extracted_files:
                    local_path = os.path.join(PATH_FOLDER, extracted_file)
                    s3_key = os.path.join(OUTPUT_DIR, extracted_file)
                    s3.upload_file(local_path, S3_BUCKET_NAME, s3_key)

        os.remove(file_name)
        logging.info(f"Download, extraction, and upload completed for {file_name}")
        
    except Exception as e:
        logging.error(f"Error during processing: {str(e)}")

def lambda_handler(event, context):
    try:
        # Deleta os objetos dentro do bucket
        objects_to_delete = [{'Key': obj.key} for obj in bucket.objects.filter(Prefix=OUTPUT_DIR)]
        if objects_to_delete:
            logging.info(objects_to_delete)
            s3.delete_objects(Bucket=S3_BUCKET_NAME, Delete={'Objects': objects_to_delete})
        
        for i in range(5):
            file_url = f'{url_base}Estabelecimentos{i}.zip'
            file_name = os.path.join(PATH_FOLDER, f'Estabelecimentos{i}.zip')
            extract_and_upload_file(file_url, file_name)
        
        return {
            'statusCode': 200,
            'body': json.dumps('Process completed successfully!')
        }
    except Exception as e:
        logging.error(f"An error occurred: {str(e)}")
        return {
            'statusCode': 500,
            'body': json.dumps('An error occurred during processing.')
        }


In [ ]:
def calculo_daora(x,y):
    if y == 0:
        return 0
    elif y == 1:
        return x
    else : 
        return x +calculo_daora(x,y-1)

valor = calculo_daora(2,500)
valor

In [ ]:
def requisicao(uf:str="MG"):
    import requests
    # Request da api na sensedia
    url = "https://api.abrasel.com.br/dev.api/v1/estabelecimentos"
    client_id = {"client_id": "b78911f9-68ad-4b71-aca8-8dc21237b506"}
    params = {
        "cidade":"ARCOS",
        "uf":uf,
        "associados":"ATIVO",
        "bandeira" :"ticket",
        "pageSize":"50"
    }

    response = requests.get(url=url, headers=client_id, params=params)

    print(response.text)

In [ ]:
UFS = ["SP", "MG", "PE", "RO", "AC", "RJ"]
import time 
for uf in UFS:
    time.sleep(1)
    requisicao(uf=uf)